# Hierarchically Chaining the Agent Calls using a Router Agent

## 8.1. Start Up both ACP Servers

First make sure the Insurer server is still running:
- then you can run the server again by typing:
  - `uv run crew_agent_server.py`

In [1]:
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/1", width=800, height=600)

Also make sure the Hospital server is still running:
- then you can run the server again by typing:
  - `uv run smolagents_server.py`

In [2]:
IFrame(f"{url}terminals/2", width=800, height=600)

## 8.2. Import ACPCallingAgent

The router agent is already implemented for you as the `ACPCallingAgent`. You are provided with a python file called `fastacp.py` where you can find the definition of the `ACPCallingAgent`. 

In [3]:
import asyncio 
import nest_asyncio
from acp_sdk.client import Client
from smolagents import LiteLLMModel
from fastacp import AgentCollection, ACPCallingAgent
from colorama import Fore

In [4]:
print(ACPCallingAgent.__doc__)


    This agent uses JSON-like ACP agent calls, similarly to how ToolCallingAgent uses tool calls,
    but directed at remote ACP agents instead of local tools.
    
    Args:
        acp_agents (`dict[str, Agent]`): ACP agents that this agent can call.
        model (`Callable[[list[dict[str, str]]], ChatMessage]`): Model that will generate the agent's actions.
        prompt_templates ([`Dict[str, str]`], *optional*): Prompt templates.
        planning_interval (`int`, *optional*): Interval at which the agent will run a planning step.
        **kwargs: Additional keyword arguments.
    


## 8.3. Run the Hierarchical Workflow 

In [5]:
nest_asyncio.apply()

In [6]:
model = LiteLLMModel(
    model_id="openai/gpt-4"
)

async def run_hospital_workflow() -> None:
    async with Client(base_url="http://localhost:8001") as insurer, Client(base_url="http://localhost:8000") as hospital:
        # agents discovery
        agent_collection = await AgentCollection.from_acp(insurer, hospital)  
        acp_agents = {agent.name: {'agent':agent, 'client':client} for client, agent in agent_collection.agents}
        print(acp_agents) 
        # passing the agents as tools to ACPCallingAgent
        acpagent = ACPCallingAgent(acp_agents=acp_agents, model=model)
        # running the agent with a user query
        result = await acpagent.run("do i need rehabilitation after a shoulder reconstruction and what is the waiting period from my insurance?")
        print(Fore.YELLOW + f"Final result: {result}" + Fore.RESET)

In [ ]:
asyncio.run(run_hospital_workflow())